the ipynb is refer: 
Create the premodel by ResNet50,Xception, InceptionV3, VGG16, VGG19

In [ ]:
write_pretrain_data = False

import os
import shutil

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py
import math

dir = "/ext/Data/distracted_driver_detection/"

def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    print(MODEL.__name__)
    gen = ImageDataGenerator()
    batch_size = 16
    train_generator = gen.flow_from_directory(os.path.join(dir, 'train'), image_size, shuffle=False, batch_size=batch_size)
    test_generator = gen.flow_from_directory(os.path.join(dir, 'test'), image_size, shuffle=False, batch_size=batch_size, class_mode=None)

    print("predict_generator train {}".format(math.ceil(train_generator.samples/batch_size)))n
    train = model.predict_generator(train_generator, math.ceil(train_generator.samples/batch_size))
    print("train: {}".format(train.shape))
    print("predict_generator test {}".format(math.ceil(test_generator.samples/batch_size)))
    test = model.predict_generator(test_generator, math.ceil(test_generator.samples/batch_size))
    print("test: {}".format(test.shape))
    print("label: {}".format(train_generator.classes.shape))

    print("begin create database {}".format(Model.__name__))
    with h5py.File("models/bottleneck_%s.h5" % MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)
    print("write_gap {} successed".format(Model.__name__))
    
if write_pretrain_data:
    write_gap(ResNet50, (224, 224))
    write_gap(Xception, (299, 299))
    write_gap(InceptionV3, (299, 299))
    write_gap(VGG16, (224, 224))
    write_gap(VGG19, (224, 224))

In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle

np.random.seed(2017)

def one_hot_encode(y):
    l = list()
    for item in y:
        c = [0. for i in range(10)]
        c[item] = 1.
        l.append(c)
    return np.array(l)

def load_data(pretrain_data_file):
    print('load data: ' + pretrain_data_file)
    with h5py.File("models/" + pretrain_data_file, 'r') as h:
        X_train = np.array(h['train'])
        X_valid = np.array(h['valid'])
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])
        X_train, y_train = shuffle(X_train, y_train)
        y_train = one_hot_encode(y_train)
        X_valid, y_valid = shuffle(X_valid, y_valid)
        y_valid = one_hot_encode(y_valid)
        return X_train, y_train, X_valid, y_valid
    
from keras.models import *
from keras.layers import *

def make_model(input_shape):

#     input_tensor = Input(shape=input_shape)
#     #x = Flatten()(inp)
#     x = Dropout(0.5)(input_tensor)
#     x = Dense(10, activation='softmax')(x)
#     model = Model(input_tensor, x)
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    input_tensor = Input(input_shape)
    x = input_tensor
#    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(input_tensor, x)
    model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model
print("Done")

Using TensorFlow backend.


Done


ResNet50

In [2]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_ResNet50.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_valid,y_valid))

load data: bottleneck_ResNet50.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 0.2550 - acc: 0.9075 - val_loss: 0.2571 - val_acc: 0.9039
Epoch 2/30
20787/20787 [==============================] - ETA: 0s - loss:

Xception

In [4]:
X_train, y_train, X_vaiid, y_valid = load_data("bottleneck_Xception.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_valid,y_valid))

load data: bottleneck_Xception.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 0.2804 - acc: 0.9004 - val_loss: 0.6166 - val_acc: 0.8209
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.21

InceptionV3

In [5]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_InceptionV3.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_valid,y_valid))

load data: bottleneck_InceptionV3.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 0.2628 - acc: 0.9033 - val_loss: 0.2591 - val_acc: 0.9053
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0

VGG16

In [6]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_VGG16.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_valid,y_valid))

load data: bottleneck_VGG16.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 0.6627 - acc: 0.8526 - val_loss: 0.9816 - val_acc: 0.8407
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.3276 - acc: 0.8976 - val_loss: 0.7966 - val_acc: 0.8610

VGG19

In [7]:
X_train, y_train, X_valid, y_valid = load_data("bottleneck_VGG19.h5")
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_valid,y_valid))

load data: bottleneck_VGG19.h5
Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 0.5918 - acc: 0.8553 - val_loss: 0.7854 - val_acc: 0.8526
Epoch 2/30
20787/20787 [==============================] - 0s - loss: 0.3058 - acc: 0.8990 - val_loss: 0.6517 - val_acc: 0.8647

combine  ResNet50,Xception and InceptionV3

In [11]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(2017)

X_train = []
X_valid = []

for filename in ["bottleneck_ResNet50.h5", "bottleneck_InceptionV3.h5", "bottleneck_VGG16.h5", "bottleneck_VGG19.h5"]:
    print('------------------'+filename)
    with h5py.File("models/" + filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

X_train, y_train = shuffle(X_train, y_train)
y_train = one_hot_encode(y_train)
X_valid, y_valid = shuffle(X_valid, y_valid)
y_valid = one_hot_encode(y_valid)

print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

------------------bottleneck_ResNet50.h5
------------------bottleneck_InceptionV3.h5
------------------bottleneck_VGG16.h5
------------------bottleneck_VGG19.h5
(20787, 5120)
(20787, 10)
(1637, 5120)
(1637, 10)


In [14]:
model = make_model(X_train.shape[1:])
model.fit(X_train, y_train, batch_size=128, epochs=30, validation_data=(X_valid,y_valid))
model.save("models/model.h5")
print("model save successed")

Train on 20787 samples, validate on 1637 samples
Epoch 1/30
20787/20787 [==============================] - 0s - loss: 0.2683 - acc: 0.9134 - val_loss: 0.2809 - val_acc: 0.9114

In [15]:
y_pred = model.predict(X_kaggle, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)
print()
print(y_pred[:1])

import pandas as pd
from keras.preprocessing.image import *

dir = "/ext/Data/distracted_driver_detection/"

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(dir + "test/", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

l = list()
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:]
    l.append( [name, *y_pred[i]] )
    #index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    #df.set_value(index-1, 'label', y_pred[i])
    #df.append(df2, ignore_index=True
l = np.array(l)
data = {'img': l[:,0]}
for i in range(10):
    data["c%d"%i] = l[:,i+1]
df = pd.DataFrame(data, columns=['img'] + ['c%d'%i for i in range(10)])
df.head(10)
df = df.sort_values(by='img')
df.to_csv('pred.csv', index=None, float_format='%.3f')

78816/79726 [============================>.] - ETA: 0s  